In [11]:
from neo4j import GraphDatabase
import cred_neo4j as c
import pandas as pd

In [12]:
driver = GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password))

In [13]:
def resetDb():
    q1 = """
    match (n) detach delete n
    """

    q2 = """
    create 
      // ----------------------------------------
      // Admins, Gruppen, Gruppenstruktur
      // ----------------------------------------
      (a1:Admin {name: 'a1'}),
      (a2:Admin {name: 'a2'}),
      (g1:Gruppe {name: 'g1'}),
      (g2:Gruppe {name: 'g2'}),
      (g3:Gruppe {name: 'g3'}),
      (a1)-[:ING]->(g1),
      (a1)-[:ING]->(g3),
      (a2)-[:ING]->(g2),
      (a2)-[:ING]->(g3),
      // ----------------------------------------
      // Unternehmen, Unternehmenstrutur
      // ----------------------------------------
      (u1:Unternehmen {name: 'u1'}),
      (u11:Unternehmen {name: 'u11'}),
      (u111:Unternehmen {name: 'u111'}),
      (u112:Unternehmen {name: 'u112'}),
      (u1111:Unternehmen {name: 'u1111'}),
      (u1112:Unternehmen {name: 'u1112'}),
      (u12:Unternehmen {name: 'u12'}),
      (u121:Unternehmen {name: 'u121'}),
      (u122:Unternehmen {name: 'u122'}),
      (u11)-[:SUB]->(u1),
      (u12)-[:SUB]->(u1),
      (u111)-[:SUB]->(u11),
      (u1111)-[:SUB]->(u111),
      (u1112)-[:SUB]->(u111),
      (u112)-[:SUB]->(u11),
      (u121)-[:SUB]->(u12),
      (u122)-[:SUB]->(u12)
    """

    with driver.session() as session:
        session.run(q1)
        session.run(q2)


# Lookup Tabelle um Rechte zu vergleichen, was ist möglich wenn Auth X gesetzt wurde?
rights_lookup = {
    "AI": ["AI", "DI", "A", "D"],
    "DI": ["DI", "A", "D"],
    "A": ["A", "D"],
    "D": []
}


# Erhalte nutzer in der gegebenen gruppe
def get_users_in_group(gruppe):
    with driver.session() as session:
        query = f"""
        MATCH (g:Gruppe {{name: '{gruppe}'}})<-[:ING]-(a:Admin)
        RETURN a.name;
        """
        res = session.run(query)
        return pd.DataFrame(res.data())


def setAuth(gruppe, unternehmen, auth):
    with driver.session() as session:
        df_group = pd.DataFrame(get_users_in_group(gruppe))

        # Teste ob Auth Anweisung mehr als ein Buchstaben hat
        # Wenn ja, muss Pfad variabler Länge genutzt werden.
        if len(auth) > 1:
            # Iteriere über gefundene Admins
            for index, row in df_group.iterrows():
                admin_name = row['a.name']

                # sammle derzeitige berechtigungen für benutzer
                query = f"""
                    MATCH (a:Admin {{name: '{admin_name}'}})-[acc:ACCS]->(u:Unternehmen)
                    RETURN a.name as admin, acc.val as accesstype, u.name as access
                """
                res = session.run(query)
                df_accessrecords = pd.DataFrame(res.data())
                #print(f"\nBestehende Rechte für admin {admin_name}:\n{df_accessrecords}")

                # Erstelle neue Auth Beziehung wenn nicht vorhanden
                query = f"""
                    MATCH (h1:Unternehmen {{name: '{unternehmen}'}})<-[:SUB*0..]-(sub:Unternehmen)
                    WITH DISTINCT sub AS subs
                
                    MATCH (a:Admin {{name: '{row['a.name']}'}})
                    UNWIND subs AS sub
                
                    MERGE (a)-[acc:ACCS]->(sub)
                    ON CREATE SET acc.val = '{auth}'
                """
                session.run(query)

                # Sammle alle bestehenden Beziehungen
                query = f"""
                    MATCH (u:Unternehmen {{name: '{unternehmen}'}})<-[:SUB*0..]-(sub:Unternehmen)
                    WITH DISTINCT sub AS subs

                    MATCH (a:Admin {{name: '{admin_name}'}})-[acc:ACCS]->(sub)
                    UNWIND subs AS sub2

                    RETURN sub2.name AS access, acc.val AS accesstype
                """
                res = session.run(query)
                df_current_auth = pd.DataFrame(res.data())
                
                # Iteriere über alle bestehenden Beziehungen
                for index, row in df_current_auth.iterrows():
                    # Teste ob bestehender Auth gemäß Lookup Tabelle überschrieben werden darf
                    if auth in rights_lookup[row['accesstype']]:
                        query = f"""
                            MATCH (a:Admin {{name: '{admin_name}'}})-[acc:ACCS]->(u:Unternehmen {{name: '{row['access']}'}})
                            SET acc.val = '{auth}'
                        """
                        session.run(query)

        # Wenn nein, muss nur einzelne Beziehung angelegt oder aktualisiert werden
        else:
            for index, row in df_group.iterrows():
                admin_name = row['a.name']
                query = f"""
                    MATCH (a:Admin {{name: '{admin_name}'}})-[acc:ACCS]->(u:Unternehmen {{name: '{unternehmen}'}})
                    RETURN a.name as admin, acc.val as accesstype, u.name as access
                """
                res = session.run(query)
                df_accessrecords = pd.DataFrame(res.data())

                if df_accessrecords.empty:
                    query = f"""
                        MATCH (a:Admin {{name: '{admin_name}'}})-[acc:ACCS]->(u:Unternehmen {{name: '{unternehmen}'}})
                        MERGE (a)-[:ACCS {{val: '{auth}'}}]->(u) 
                    """
                    session.run(query)

                else:
                    for index, row in df_accessrecords.iterrows():
                        # teste ob existierender auth ueberschrieben werden darf
                        if auth in rights_lookup[row['accesstype']]:
                            query = f"""
                                MATCH (a:Admin {{name: '{admin_name}'}})-[acc:ACCS]->(u:Unternehmen {{name: '{unternehmen}'}})
                                SET acc.val = '{auth}'
                            """
                            session.run(query)


def unsetAuth(gruppe, unternehmen, auth):
    with driver.session() as session:
        df_admins = pd.DataFrame(get_users_in_group(gruppe))

        if len(auth) > 1:
            for index, row in df_admins.iterrows():
                admin_name = row['a.name']
                query = f"""
                    MATCH (u:Unternehmen {{name: '{unternehmen}'}})<-[:SUB*0..]-(sub:Unternehmen)
                    WITH DISTINCT sub AS subs

                    MATCH (a:Admin {{name: '{admin_name}'}})-[acc:ACCS {{val: '{auth}'}}]->(sub)

                    DELETE acc
                """
                session.run(query)

        else:
            for index, row in df_admins.iterrows():
                admin_name = row['a.name']
                query = f"""
                    MATCH (a:Admin {{name: '{admin_name}'}})-[acc:ACCS {{val: '{auth}'}}]->(u:Unternehmen {{name: '{unternehmen}'}})
                    DELETE acc
                """
                session.run(query)


def showAuth(debug=False):
    with driver.session() as session:
        # erhalte alle beziehungen wo wert nicht D oder DI ist
        if debug:
            query = """
                MATCH (a:Admin)-[acc:ACCS]->(u:Unternehmen)
                RETURN a.name as admin, u.name as access, acc.val as accesstype
                ORDER BY admin, access;
            """

        else:
            query = """
                MATCH (a:Admin)-[acc:ACCS]->(u:Unternehmen)
                WHERE acc.val IN ['A', 'AI']
                RETURN a.name as admin, u.name as access
                ORDER BY admin, access;
            """
        res = session.run(query)
        df = pd.DataFrame(res.data())
        print(f"\nDerzeit gültige Rechte ('A' oder 'AI'):\n{df}")


In [14]:
resetDb()

In [15]:
# Beispiel 1
setAuth("g3", "u1", "AI")
showAuth()
#unsetAuth("g3", "u1", "AI")
#showAuth()


Derzeit gültige Rechte ('A' oder 'AI'):
   admin access
0     a1     u1
1     a1    u11
2     a1   u111
3     a1  u1111
4     a1  u1112
5     a1   u112
6     a1    u12
7     a1   u121
8     a1   u122
9     a2     u1
10    a2    u11
11    a2   u111
12    a2  u1111
13    a2  u1112
14    a2   u112
15    a2    u12
16    a2   u121
17    a2   u122


In [ ]:
# Beispiel 2
setAuth("g1", "u12", "DI")
setAuth("g2", "u11", "DI")
#unsetAuth("g1", "u12", "DI")
#unsetAuth("g2", "u11", "DI")
#showAuth(True)
showAuth()

In [7]:
# Beispiel 3
setAuth("g1", "u111", "DI")
# unsetAuth("g1", "u111", "DI")
showAuth()


Derzeit gültige Rechte ('A' oder 'AI'):
  admin access
0    a1     u1
1    a1    u11
2    a1   u112
3    a2     u1
4    a2    u12
5    a2   u121
6    a2   u122


In [8]:
# Beispiel 4
setAuth("g1", "u1111", "A")
# unsetAuth("g1", "u1111", "A")
showAuth()


Derzeit gültige Rechte ('A' oder 'AI'):
  admin access
0    a1     u1
1    a1    u11
2    a1  u1111
3    a1   u112
4    a2     u1
5    a2    u12
6    a2   u121
7    a2   u122


In [9]:
# Beispiel 5
setAuth("g2", "u12", "D")
# unsetAuth("g2", "u12", "D")
showAuth()


Derzeit gültige Rechte ('A' oder 'AI'):
  admin access
0    a1     u1
1    a1    u11
2    a1  u1111
3    a1   u112
4    a2     u1
5    a2   u121
6    a2   u122


In [10]:
# Beispiel 6
setAuth("g3", "u1", "D")
# unsetAuth("g3", "u1", "D")
showAuth()


Derzeit gültige Rechte ('A' oder 'AI'):
  admin access
0    a1    u11
1    a1  u1111
2    a1   u112
3    a2   u121
4    a2   u122
